In [12]:
import requests
import tensorflow as tf
import zipfile,os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D,Dropout
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from PIL import Image
%matplotlib inline


In [13]:
dir= "fitur 1"

In [36]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,  # Rotasi gambar dalam rentang 20 derajat
                    width_shift_range=0.2,  # Pergeseran horizontal gambar sebesar 20% dari lebar gambar
                    height_shift_range=0.2,  # Pergeseran vertikal gambar sebesar 20% dari tinggi gambar
                    shear_range=0.2,  # Shearing gambar dengan rentang 20 derajat
                    zoom_range=0.2,  # Perbesaran dan pengecilan gambar sebesar 20%
                    horizontal_flip=True,  # Flipping gambar secara horizontal
                    fill_mode='nearest' ,# Mengisi pixel yang kosong dengan pixel terdekat  
                    validation_split = 0.4)

In [37]:
train_generator = train_datagen.flow_from_directory(
        dir,  # direktori data latih
        target_size=(100, 150),  # mengubah resolusi seluruh gambar menjadi 100x150 piksel
        shuffle =True,
        class_mode='categorical',
        subset= 'training'
        )

validation_generator = train_datagen.flow_from_directory(
        dir, # direktori data validasi
        target_size=(100, 150), # mengubah resolusi seluruh gambar menjadi 100x150 piksel
        class_mode='categorical',
        subset= 'validation'
        )  

Found 720 images belonging to 8 classes.
Found 480 images belonging to 8 classes.


In [20]:
# wrap the generators with RepeatDataset
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, *(100, 150, 3)], [None, 8])
).repeat()

val_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, *(100, 150, 3)], [None, 8])
).repeat()

In [38]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(100, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(8, activation='softmax') 
])



In [113]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])


class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.7):
      print("\nAkurasi telah mencapai >70%!")
      self.model.stop_training = True
callbacks = myCallback()


In [114]:
hist = model.fit(
    train_generator,
    steps_per_epoch= 16,  
    epochs= 50,  
    validation_data=validation_generator,
    validation_steps= 12,  
    verbose=1,  
    batch_size= 16,  
    callbacks=[callbacks]
)


Epoch 1/50
16/16 [==============================] - 20s 1s/step - loss: 0.3203 - accuracy: 0.8871 - val_loss: 1.8464 - val_accuracy: 0.6380
Epoch 2/50
16/16 [==============================] - 23s 1s/step - loss: 0.2124 - accuracy: 0.9121 - val_loss: 1.7482 - val_accuracy: 0.6302
Epoch 3/50
16/16 [==============================] - 32s 2s/step - loss: 0.2552 - accuracy: 0.9023 - val_loss: 1.5995 - val_accuracy: 0.6380
Epoch 4/50
16/16 [==============================] - 31s 2s/step - loss: 0.2684 - accuracy: 0.9254 - val_loss: 1.8285 - val_accuracy: 0.6224
Epoch 5/50
16/16 [==============================] - 31s 2s/step - loss: 0.2057 - accuracy: 0.9194 - val_loss: 1.6652 - val_accuracy: 0.6484
Epoch 6/50
16/16 [==============================] - 24s 1s/step - loss: 0.1879 - accuracy: 0.9375 - val_loss: 1.7625 - val_accuracy: 0.6354
Epoch 7/50
16/16 [==============================] - 17s 1s/step - loss: 0.1904 - accuracy: 0.9375 - val_loss: 1.7805 - val_accuracy: 0.6120
Epoch 8/50
16/16 [==